# Modello:
$ M_a = \frac{1}{n} \sum_{i=1}^{n} x_i $

In [1]:
import os
from binance.spot import Spot
import time
import pandas as pd
import numpy as np
# import talib

In [ ]:
os.system('wget https://github.com/TA-Lib/ta-lib/releases/download/v0.4.0/ta-lib-0.4.0-src.tar.gz')
os.system('tar -xf ta-lib-0.4.0-src.tar.gz')
os.chdir('ta-lib/')
os.system('bash configure --prefix=/usr/')
os.system('make')
os.system('sudo make install')
os.system('pip install TA-Lib')
os.chdir('/content/')

import talib

## Parametri

In [34]:
start = "2024-05-11 13:00"
end = '2024-05-13 13:00'
window1 = 5
window2 = 21

## Backtest

In [35]:
start_obj = time.strptime(start, '%Y-%m-%d %H:%M')
start_ts = time.mktime(start_obj)
start_int = int(start_ts)
start_ts_ms = start_int * 1000

end_obj = time.strptime(end, '%Y-%m-%d %H:%M')
end_ts = time.mktime(end_obj)
end_int = int(end_ts)
end_ts_ms = end_int * 1000

spot_client = Spot(base_url = 'https://testnet.binance.vision', api_key = 'OMISSIS', api_secret = 'OMISSIS')

def dati_backtest():
    global c_df
    a = spot_client.klines('BTCUSDT', '1m', startTime = start_ts_ms, endTime = end_ts_ms)
    c_df = pd.DataFrame(a, columns = ['open time', 'open', 'high', 'low', 'close', 'volume', 'close time', 'quote asset volume', 'number of trades', 'taker buy base asset volume', 'taker buy quote asset volume', 'unused'])
    
    
def modello():
    global l, m, y, c_df
    c_df['x'] = talib.SMA(c_df.close, timeperiod = window1)
    c_df['y'] = talib.SMA(c_df.close, timeperiod = window2)
    c_df['x'] = c_df['x'].fillna(0)
    c_df['y'] = c_df['y'].fillna(0)
    l = l + 1
    if x == 0:
        y = x
    if x > 0:
        y = x - 1
    # se x > y e x < y a t-1 compra
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 1
            c_df['e'].loc[x] = l
    # se x > y e x > y a t-1 neutro
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0
    # se x < y e x > y a t-1 vende
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:  
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = -1
            c_df['e'].loc[x] = l
    # se x < y e x < y a t-1 neutro
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0

def backtest():
    global n, o, c_df
    n = 0
    o = 0
    if c_df['d'].loc[x] == 1:
        c_df['w'].loc[x] = c_df['close'].loc[x]
    if c_df['d'].loc[x] == -1:
        c_df['z'].loc[x] = c_df['close'].loc[x]
    c_df['w'] = c_df['w'].fillna(0)
    c_df['z'] = c_df['z'].fillna(0)
    c_df = c_df.astype({'w': 'float', 'z': 'float'})
    for p in c_df['w']:
        n += p
    for p in c_df['z']:
        o += p

dati_backtest()        

l = 0
m = 0
c_df['d'] = 0
c_df['e'] = 0
c_df['f'] = 0
c_df['h'] = 0
c_df['i'] = 0
c_df['w'] = 0
c_df['z'] = 0       

for x in range(len(c_df)):
    modello()
    backtest()
    
print('Comprato:')
print(n)
print('Venduto:')
print(o)

Comprato:
1706917.56
Venduto:
1707487.28


## Operazioni

In [ ]:
spot_client = Spot(base_url = 'https://testnet.binance.vision', api_key = 'OMISSIS', api_secret = 'OMISSIS')

# da sistemare posizione dichiarazione variabili
c_df = pd.DataFrame()
l = 0
m = 0
c_df['d'] = 0
c_df['e'] = 0
c_df['f'] = 0
c_df['h'] = 0
c_df['i'] = 0
c_df['w'] = 0
c_df['z'] = 0

def dati():
    global c_df
    a = spot_client.klines('BTCUSDT', '1m', limit=1)
    b_df = pd.DataFrame(a, columns = ['open time', 'open', 'high', 'low', 'close', 'volume', 'close time', 'quote asset volume', 'number of trades', 'taker buy base asset volume', 'taker buy quote asset volume', 'unused'])
    c_df = pd.concat([c_df, b_df])
    
def modello():
    global l, m, y, c_df
    c_df['x'] = talib.SMA(c_df.close, timeperiod = window1)
    c_df['y'] = talib.SMA(c_df.close, timeperiod = window2)
    c_df['x'] = c_df['x'].fillna(0)
    c_df['y'] = c_df['y'].fillna(0)
    l = l + 1
    if x == 0:
        y = x
    if x > 0:
        y = x - 1
    # se x > y e x < y a t-1 compra
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 1
            c_df['e'].loc[x] = l
    # se x > y e x > y a t-1 neutro
    if c_df['x'].loc[x] > c_df['y'].loc[x]:
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0
    # se x < y e x > y a t-1 vende
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:  
        if c_df['x'].loc[y] > c_df['y'].loc[y]:
            c_df['d'].loc[x] = -1
            c_df['e'].loc[x] = l
    # se x < y e x < y a t-1 neutro
    if c_df['x'].loc[x] <=  c_df['y'].loc[x]:
        if c_df['x'].loc[y] <=  c_df['y'].loc[y]:
            c_df['d'].loc[x] = 0

def operazioni():
    

for x in range(0, 60):
    dati()
    modello()
    operazioni()